In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
#!pip install sklearn-xarray

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from math import radians, sin, cos, sqrt, atan2

from google.colab import files

In [ ]:
%cd drive/MyDrive/BTTAI_Project_Team_B/final_model/

/content/drive/.shortcut-targets-by-id/18IWwVNOjqLhXRJD-X_E3ABiSdXm6rkqB/BTTAI_Project_Team_B/final_model


In [ ]:
DataSet = xr.open_dataset("/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/final_merged_dataset.nc")

subset_da = DataSet.sel(lat=slice(24.396308, 49.384358), lon=slice(-124.848974, -66.885444),year=slice(2006,2024))
df = subset_da.to_dataframe()
df.reset_index(inplace=True)
df.dropna(subset=['maxHuss'], inplace=True)
df = df.replace(to_replace='', value=np.nan)
#df = df.dropna(subset='null_variable')
df['wns'] = df['null_variable'].map(lambda x: 1 if x == 'W' else 0)
df

,lon,lat,year,null_variable,maxHuss,maxPrec,maxTemp,meanHuss,meanPrec,meanRsds,meanTemp,minHuss,minPrec,minTemp,wns
1786,-124.625,48.125,2006,NaN,0.009661,93.080559,23.765198,0.005861,7.369194,129.868134,10.309885,0.002125,0.0,-4.571747,0
1787,-124.625,48.125,2007,NaN,0.010276,71.963501,21.892517,0.006272,7.148562,134.327148,9.157192,0.002941,0.0,-1.255707,0
1788,-124.625,48.125,2008,NaN,0.010546,79.569580,18.406464,0.006400,10.171446,127.175758,9.170335,0.003059,0.0,0.045380,0
1789,-124.625,48.125,2009,NaN,0.010493,116.354836,23.448608,0.005665,10.867501,121.978271,9.798295,0.001608,0.0,-5.656860,0
1790,-124.625,48.125,2010,NaN,0.009748,77.657959,21.894501,0.005843,9.717582,127.913719,10.851817,0.003021,0.0,1.998596,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438857,-67.125,48.875,2020,NaN,0.014111,35.291557,29.737671,0.004503,3.454184,152.088226,5.040043,0.000280,0.0,-22.328613,0
438858,-67.125,48.875,2021,NaN,0.013859,49.752010,25.446899,0.004673,4.035824,156.414658,4.608527,0.000285,0.0,-21.159241,0
438859,-67.125,48.875,2022,NaN,0.014111,52.268139,29.762695,0.004435,3.753445,156.895508,5.123062,0.000284,0.0,-20.920166,0
438860,-67.125,48.875,2023,NaN,0.012564,43.042122,26.450562,0.004270,4.023201,153.472992,4.619852,0.000310,0.0,-25.559784,0


In [ ]:


# Sort DataFrame by Year in ascending order
df = df.sort_values(by='year')

# Function to check if WNS was detected in previous years for a specific coordinate
def wns_detected_past_years(row):
    past_years_data = df[(df['lat'] == row['lat']) & (df['lon'] == row['lon']) & (df['year'] < row['year'])]
    if len(past_years_data) > 0 and (past_years_data['wns'] == 1).any():
        return 1  # WNS detected in past years for this coordinate
    else:
        return 0  # WNS not detected in past years for this coordinate

# Apply the function to create a new column 'WNS_Detected_Past_Years'
df['WNS_Detected_Past_Years'] = df.apply(wns_detected_past_years, axis=1)



In [ ]:
# Function to calculate distance between two coordinates using Haversine formula

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Function to calculate shortest distance for each point to a coordinate with previous WNS detection
def shortest_distance_to_wns(row):
    wns_detected_coords = df[(df['WNS_Detected_Past_Years'] == 1)]
    distances = [haversine(row['lat'], row['lon'], lat, lon) for lat, lon in wns_detected_coords[['lat', 'lon']].values]
    if len(distances) > 0:
        return min(distances)
    else:
        return None

# Apply the function to create a new column 'Shortest_Distance_to_WNS'
df['Shortest_Distance_to_WNS'] = df.apply(shortest_distance_to_wns, axis=1)
df_2024 = df[df['year'] == 2024]


# Save DataFrame with predictions to a CSV file


df_2024.to_csv('df_2024.csv', index=False)

files.download('df_2024.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_2024

,lon,lat,year,null_variable,maxHuss,maxPrec,maxTemp,meanHuss,meanPrec,meanRsds,meanTemp,minHuss,minPrec,minTemp,wns,WNS_Detected_Past_Years,Shortest_Distance_to_WNS
122853,-108.625,40.875,2024,NaN,0.009897,34.657894,29.023560,0.003716,0.939433,194.187180,4.653328,0.000824,0.0,-29.550095,0,0,168.396189
184717,-100.375,29.875,2024,NaN,0.021069,51.627815,32.748932,0.009450,1.129149,200.477524,16.815100,0.000852,0.0,-9.725464,0,0,60.573802
284467,-87.375,42.375,2024,NaN,0.020434,133.951324,31.300812,0.006735,5.533653,153.670746,12.046003,0.000518,0.0,-12.185455,0,0,150.793418
207745,-97.375,32.875,2024,NaN,0.024762,51.854954,39.269409,0.010940,1.598785,180.924225,18.269894,0.001792,0.0,-7.782104,0,0,194.591122
28423,-121.125,48.375,2024,NaN,0.012779,48.532455,27.498962,0.004863,3.602767,173.825378,4.259709,0.000566,0.0,-18.529831,0,0,46.296147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369758,-76.125,39.625,2024,NaN,0.020236,165.785568,34.311829,0.007651,4.695564,158.579788,14.296403,0.001331,0.0,-7.974884,0,0,42.823059
319826,-82.625,32.625,2024,NaN,0.018958,89.590675,35.261505,0.009705,3.539485,181.727844,18.295761,0.002318,0.0,-3.525085,0,0,155.508431
92776,-112.625,45.125,2024,NaN,0.012686,17.344841,27.998657,0.003964,1.044003,172.694199,3.374712,0.000489,0.0,-25.105164,0,0,452.861124
369777,-76.125,39.875,2024,NaN,0.018976,163.533859,33.894440,0.007320,4.249761,159.985138,13.724575,0.001127,0.0,-9.429474,0,0,50.989674


In [ ]:
df_2024 = pd.open_csv("/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/data_2024.csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-0e6de184bc85>", line 1, in <cell line: 1>
    df_2024 = pd.open_csv("/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/data_2024.csv")
  File "/usr/local/lib/python3.10/dist-packages/pandas/__init__.py", line 264, in __getattr__
    raise AttributeError(f"module 'pandas' has no attribute '{name}'")
AttributeError: module 'pandas' has no attribute 'open_csv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback 

In [ ]:
df_with_predictions = df_2024.copy()  # Make a copy of the original DataFrame
df_with_predictions['Predictions'] = predictions  # Add predictions as a new column

# Save DataFrame with predictions to a CSV file
df_with_predictions.to_csv('df_with_predictions.csv', index=False)

In [ ]:
import pickle  # or import joblib

# Load the saved Random Forest model using pickle or joblib
with open('random_forest_model_2024.pkl', 'rb') as file:
    loaded_model = pickle.load(file)  # or loaded_model = joblib.load('random_forest_model.joblib')

# Example of new data (replace this with your actual new data)
new_data = df_2024

# Make predictions on the new data using the loaded model
predictions = loaded_model.predict(new_data)

# Display the predictions
#print("Predictions:")
#for pred in predictions:
#    print(pred)
predictions

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Custom TB Handler failed, unregistering
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-dd72181e4d2f>", line 4, in <cell line: 4>
    with open('random_forest_model_2024.pkl', 'rb') as file:
OSError: [Errno 107] Transport endpoint is not connected: 'random_forest_model_2024.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 281, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, co